In [ ]:
import numpy as np
from random import randint
import scipy as scipy
import scipy.interpolate as interpolate
import math
from random import *

In [ ]:
# here we generate matrix X
dimensionality = 400
k = 10
delta = 0.1 # regulates percentage of outliers
MODE = 'neutral' # case I - neutral, case II-something else
number_of_points = 400
number_of_neurons = number_of_points

A = np.random.normal(loc=0.0, scale=1.0, size=(int(number_of_points*(1-delta)), k))
B = np.random.normal(loc=0.0, scale=1.0, size=(k, dimensionality))
if MODE == 'neutral':
    C = np.random.normal(loc=0.0, scale=1.0, size=(int(number_of_points*delta), dimensionality))
else:
    C = np.tile(np.random.normal(loc=0.0, scale=1.0, size=(1, dimensionality)), (int(number_of_points*delta), 1))
images = np.concatenate((np.matmul(A, B), C), axis = 0)
print(images.shape)
u, s, vh = np.linalg.svd(np.matmul(A, B), full_matrices=True)
D = vh[0:k]
P_correct = np.matmul(np.transpose(D), D)

u, s, vh = np.linalg.svd(images, full_matrices=True)
D = vh[0:k]
P_svd = np.matmul(np.transpose(D), D)
DIFF = P_svd - P_correct
dist = np.sqrt(np.sum(DIFF*DIFF))
print("Frobenius distance from SVD till correct %0.4f" %dist)

L1 = np.sum(np.sqrt(np.sum(np.square(images-np.matmul(images, P_correct)), axis=1)))
print("L12 distance till correct %0.4f" %L1)

Data = np.float32(np.transpose(images))

In [ ]:
from kurtosis import kurtosis
from mmd import mmd
import tensorflow as tf
for i in range(10):
    for GAMMA in [0.1, 0.2, 0.4, 1, 1.3, 1.5, 2, 4, 8]:
        Q = kurtosis (images, k=10, gamma=GAMMA)
        PP=np.matmul(Q, np.transpose(Q))
        diff=PP-P_correct
        dist = np.sum(np.square(diff))
        print("kurtosis %d GAMMA=%f and dist=%f\n" % (i, GAMMA, dist))
        tf.reset_default_graph()
        
        O = mmd (images, k=10, gamma=GAMMA)
        PP=np.matmul(O, np.transpose(O))
        diff=PP-P_correct
        dist=np.sum(np.square(diff))
        print("mmd %d GAMMA=%f and dist=%f\n" % (i, GAMMA, dist))
        tf.reset_default_graph()